### Will copy http://localhost:8888/notebooks/research/Jan2019/11519b/1-15-19%20check%20co-occurance%20of%20motifs.ipynb# but now use fisher exact test instead of my makeshift o/e ratios

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from scipy.stats import fisher_exact

In [3]:
bedfile  = pd.read_table('allC9compthresh_foreground.bed', header=None, names=['chrom', 'start', 'end'])

In [6]:
DF = pd.read_table('motif_name_line5.txt')
colnames = DF.columns.values

In [7]:
motifDF =  pd.read_table('C8primaryC9secondary_motifsheet.bed', header=None, names=colnames)

In [8]:
DHSname = motifDF['#chrom'] +'-' +motifDF['min0'].values.astype(str) + '-'  +motifDF['max1'].values.astype(str)

In [9]:
DHSname_bedfile = bedfile['chrom'] +'-' +bedfile['start'].values.astype(str) + '-'  +bedfile['end'].values.astype(str) 

In [10]:
bedfile['DHSname_bedfile'] = DHSname_bedfile

In [12]:
motifDF['DHSname'] = DHSname

In [13]:
mergedDF = bedfile.merge(motifDF, left_on='DHSname_bedfile', right_on='DHSname') 

In [14]:
cluster_lookup_name = 'clusters.long.txt'
clustDF = pd.read_table(cluster_lookup_name, names=['clust_name', 'motif_name', 'sign', 'number'])
clustDF_lookup = clustDF.copy()
clustnames_ar = clustDF.clust_name.value_counts().index.values
cluster_table = []
for cluster in clustnames_ar:
    print(cluster)
    relevantcols_cut = clustDF.clust_name.values == cluster
    relevant_cols = clustDF.motif_name.values[relevantcols_cut]
    relevant_cols_match = np.intersect1d(relevant_cols, np.core.defchararray.upper( mergedDF.columns.values.astype(str)))
    fung = np.in1d(np.core.defchararray.upper( mergedDF.columns.values.astype(str)), relevant_cols_match)
    cluster_table.append(mergedDF[mergedDF.columns[fung]].sum(axis=1).values)

c278-HD/5
c25-ETS
c75-Forkhead
c265-HD/2
c90-MYC/MAX
c269-HD/3
c64.ESRRA/2
c39-SOX
c85-NR/2
c68-CREB/ATF
c91-T-box
c112-Ebox
c184-E2F/2
c113-MEIS
c49-CCAAT/CEBP
c136-EGR/KLF/Sp1
c203-AP-2/2
c36-NKX
c84-NR
c66-AP-1
c57-SOX/3
c18-POU
c283-HD/6
c29-STAT
c116-GLI
c82-GR
c134-NF-kB
c80-GATA/2
c56-SOX/2
c93-MAF
c107-POU/2
c274-HNF1/POU
c272-SOX/5
c188-RUNX/AML
c117-GLI/2
c137-Sp1
c145-IRF
c121-SIX
c59-GATA
c70.MEF2A/2
c101.IRX2
c42.CUX1
c62.ESRRA
c28-NFAT
c222-CTF/NF-I
c151-ZNF
c55.SRF
c31-TCF/LEF
c40.YY1
c65-NEURO/OLIG
c109-Ebox/CACCTG
c162.TP53/2
c207-RFX/3
c198.HSF1
c205-RFX
c275.LEF1
c51.MYBL1/2
c208.PAX5/2
c270-HD/4
c37.MYB
c157-IRF/2
c100-NKX/2
c170.FOXA2
c72-SOX/4
c202-AP-2
c47.RXRA::VDR
c78.ARID5A
c158-IRF/3
c61.ESR1
c46.EVI1
c193.ASCL2
c81.POU2F2
c226-NRSF
c135-KLF/Sp4
c173.DMRT1
c279.HDX
c180-TEAD/TEF
c277.LHX6
c174.GCM1/2
c125.SFPI1
c26.HINFP1
c45.FOXG1
c183.ARNT::AHR
c216.RHOX11
c118.TBP/2
c251-CTCF_Core/2
c209.PAX2
c153.GLIS2
c281.SOX14
c19.BCL6B
c276.BBX/2
c115.MZF1_1
c83.NR2E1

In [15]:
cluster_table = np.array(cluster_table)
cluster_tableDF = pd.DataFrame(cluster_table.T, columns=clustnames_ar)
for col in cluster_tableDF.columns:
    mergedDF[col] = cluster_tableDF[col].values
clustDF_lookup.set_index('motif_name', inplace=True)

In [16]:
reference_point = 'c14.MEF2A'
rowtalbe = []

testers = ['HLF_bZIP_1'.upper(), 'Gata3_secondary'.upper(), 'DUXA_homeodomain_1'.upper(), 'RHOXF1_homeodomain_3'.upper(), 'SRF_MADS_2'.upper(), 'OLIG1_bHLH_1'.upper()]

for tester in testers:
    testcluster = clustDF_lookup.loc[tester].clust_name

    
    
    N_DHS_brain = mergedDF[mergedDF[testcluster] > 0].shape[0]
    N_DHS_total = mergedDF.shape[0]
    
    fbrain = N_DHS_brain/ N_DHS_total
    
    N_DHS_Brain_InHeart = mergedDF[(mergedDF[testcluster] > 0) & (mergedDF[reference_point] > 0)].shape[0]
    
    Nheart = mergedDF[mergedDF[reference_point] >0 ].shape[0]
    
    N_DHS_Brain_InHeart_Expected = Nheart*fbrain
    
    Obs_over_expected = N_DHS_Brain_InHeart/N_DHS_Brain_InHeart_Expected
    
    row = [tester, testcluster, N_DHS_brain, N_DHS_total, fbrain, Nheart, N_DHS_Brain_InHeart_Expected, N_DHS_Brain_InHeart, Obs_over_expected]
    rowtalbe.append(row)

    print(row)
rowtalbeDF = pd.DataFrame(rowtalbe, columns = ['brain motif', 'brain cluster', 'N_31/30 BraiCluster', 'N_31/30 Brain (total)', 'f_brain', 'N_31/30 Heartcluster', 'N_31/30 Heart+Brain_exppected', 'N_31/30 Heart+Brain', 'O/E']  )


['HLF_BZIP_1', 'c49-CCAAT/CEBP', 87, 2840, 0.03063380281690141, 50, 1.5316901408450705, 0, 0.0]
['GATA3_SECONDARY', 'c280-GATA3', 16, 2840, 0.005633802816901409, 50, 0.28169014084507044, 0, 0.0]
['DUXA_HOMEODOMAIN_1', 'c283-HD/6', 63, 2840, 0.022183098591549297, 50, 1.109154929577465, 3, 2.7047619047619045]
['RHOXF1_HOMEODOMAIN_3', 'c269-HD/3', 179, 2840, 0.0630281690140845, 50, 3.1514084507042255, 6, 1.9039106145251397]
['SRF_MADS_2', 'c55.SRF', 70, 2840, 0.02464788732394366, 50, 1.232394366197183, 1, 0.8114285714285715]
['OLIG1_BHLH_1', 'c65-NEURO/OLIG', 54, 2840, 0.019014084507042252, 50, 0.9507042253521126, 3, 3.155555555555556]


In [17]:
rowtalbeDF

,brain motif,brain cluster,N_31/30 BraiCluster,N_31/30 Brain (total),f_brain,N_31/30 Heartcluster,N_31/30 Heart+Brain_exppected,N_31/30 Heart+Brain,O/E
0,HLF_BZIP_1,c49-CCAAT/CEBP,87,2840,0.030634,50,1.531690,0,0.000000
1,GATA3_SECONDARY,c280-GATA3,16,2840,0.005634,50,0.281690,0,0.000000
2,DUXA_HOMEODOMAIN_1,c283-HD/6,63,2840,0.022183,50,1.109155,3,2.704762
3,RHOXF1_HOMEODOMAIN_3,c269-HD/3,179,2840,0.063028,50,3.151408,6,1.903911
4,SRF_MADS_2,c55.SRF,70,2840,0.024648,50,1.232394,1,0.811429
5,OLIG1_BHLH_1,c65-NEURO/OLIG,54,2840,0.019014,50,0.950704,3,3.155556


In [20]:
firstfishtest = fisher_exact([[3,47 ], [60, 2840-(63+50 - 3)]])

In [ ]:
3 + 47

In [21]:
firstfishtest

(2.904255319148936, 0.09773789243232836)

In [22]:
secondfishtest = fisher_exact([[6,50-6 ], [179-6, 2840-(179+50-6)]])

In [23]:
secondfishtest

(2.062795585916973, 0.1288264927789547)

In [29]:
reference_point = 'c14.MEF2A'
rowtalbe = []

testers = ['HLF_bZIP_1'.upper(), 'Gata3_secondary'.upper(), 'DUXA_homeodomain_1'.upper(), 'RHOXF1_homeodomain_3'.upper(), 'SRF_MADS_2'.upper(), 'OLIG1_bHLH_1'.upper()]
print('heart Y/Y   N/Y')
print('      Y/N   N/N')
print('        brain')
for tester in testers:


    testcluster = clustDF_lookup.loc[tester].clust_name

    print(reference_point, testcluster)

    
    N_DHS_brain = mergedDF[mergedDF[testcluster] > 0].shape[0]
    N_DHS_total = mergedDF.shape[0]
    Nheart = mergedDF[mergedDF[reference_point] >0 ].shape[0]
        
    N_DHS_Brain_InHeart = mergedDF[(mergedDF[testcluster] > 0) & (mergedDF[reference_point] > 0)].shape[0]
    

    print('     '+str(N_DHS_Brain_InHeart)+'   '+str(Nheart-N_DHS_Brain_InHeart ))
    print('     '+str(N_DHS_brain- N_DHS_Brain_InHeart)+'   '+str(N_DHS_total - N_DHS_brain - Nheart + N_DHS_Brain_InHeart))
    myfishtest = fisher_exact([[N_DHS_Brain_InHeart,Nheart-N_DHS_Brain_InHeart  ], [N_DHS_brain- N_DHS_Brain_InHeart, N_DHS_total - N_DHS_brain - Nheart + N_DHS_Brain_InHeart]])
    print('p =',myfishtest[-1])
    


heart Y/Y   N/Y
      Y/N   N/N
        brain
c14.MEF2A c49-CCAAT/CEBP
     0   50
     87   2703
p = 0.4042778331676369
c14.MEF2A c280-GATA3
     0   50
     16   2774
p = 1.0
c14.MEF2A c283-HD/6
     3   47
     60   2730
p = 0.09773789243232836
c14.MEF2A c269-HD/3
     6   44
     173   2617
p = 0.1288264927789547
c14.MEF2A c55.SRF
     1   49
     69   2721
p = 1.0
c14.MEF2A c65-NEURO/OLIG
     3   47
     51   2739
p = 0.06804270132323588


In [33]:
reference_point = 'c70.MEF2A/2'
rowtalbe = []

testers = ['HLF_bZIP_1'.upper(), 'Gata3_secondary'.upper(), 'DUXA_homeodomain_1'.upper(), 'RHOXF1_homeodomain_3'.upper(), 'SRF_MADS_2'.upper(), 'OLIG1_bHLH_1'.upper()]
print('heart Y/Y   Y/N')
print('      N/Y   N/N')
print('        brain')
for tester in testers:


    testcluster = clustDF_lookup.loc[tester].clust_name

    print(reference_point, testcluster)

    
    N_DHS_brain = mergedDF[mergedDF[testcluster] > 0].shape[0]
    N_DHS_total = mergedDF.shape[0]
    Nheart = mergedDF[mergedDF[reference_point] >0 ].shape[0]
        
    N_DHS_Brain_InHeart = mergedDF[(mergedDF[testcluster] > 0) & (mergedDF[reference_point] > 0)].shape[0]
    

    print('     '+str(N_DHS_Brain_InHeart)+'   '+str(Nheart-N_DHS_Brain_InHeart ))
    print('     '+str(N_DHS_brain- N_DHS_Brain_InHeart)+'   '+str(N_DHS_total - N_DHS_brain - Nheart + N_DHS_Brain_InHeart))
    myfishtest = fisher_exact([[N_DHS_Brain_InHeart,Nheart-N_DHS_Brain_InHeart  ], [N_DHS_brain- N_DHS_Brain_InHeart, N_DHS_total - N_DHS_brain - Nheart + N_DHS_Brain_InHeart]])
    print('p =',myfishtest[-1])
    

heart Y/Y   Y/N
      N/Y   N/N
        brain
c70.MEF2A/2 c49-CCAAT/CEBP
     2   171
     85   2582
p = 0.170675160273986
c70.MEF2A/2 c280-GATA3
     2   171
     14   2653
p = 0.25440025881736666
c70.MEF2A/2 c283-HD/6
     9   164
     54   2613
p = 0.012772898225512048
c70.MEF2A/2 c269-HD/3
     15   158
     164   2503
p = 0.19459487804974596
c70.MEF2A/2 c55.SRF
     5   168
     65   2602
p = 0.6140298500091274
c70.MEF2A/2 c65-NEURO/OLIG
     4   169
     50   2617
p = 0.5683772842006201


In [34]:
reference_point = 'c14.MEF2A'
rowtalbe = []

testers = ['POU1F1_POU_1'.upper(), 'Atoh1.mouse_bHLH_1'.upper(), 'ONECUT3_CUT_1'.upper(), 'POU4F3_POU_1'.upper(), 'Pou3f4_3773.1'.upper()]
print('heart Y/Y   N/Y')
print('      Y/N   N/N')
print('        brain')
for tester in testers:


    testcluster = clustDF_lookup.loc[tester].clust_name

    print(reference_point, testcluster)

    
    N_DHS_brain = mergedDF[mergedDF[testcluster] > 0].shape[0]
    N_DHS_total = mergedDF.shape[0]
    Nheart = mergedDF[mergedDF[reference_point] >0 ].shape[0]
        
    N_DHS_Brain_InHeart = mergedDF[(mergedDF[testcluster] > 0) & (mergedDF[reference_point] > 0)].shape[0]
    

    print('     '+str(N_DHS_Brain_InHeart)+'   '+str(Nheart-N_DHS_Brain_InHeart ))
    print('     '+str(N_DHS_brain- N_DHS_Brain_InHeart)+'   '+str(N_DHS_total - N_DHS_brain - Nheart + N_DHS_Brain_InHeart))
    myfishtest = fisher_exact([[N_DHS_Brain_InHeart,Nheart-N_DHS_Brain_InHeart  ], [N_DHS_brain- N_DHS_Brain_InHeart, N_DHS_total - N_DHS_brain - Nheart + N_DHS_Brain_InHeart]])
    print('p =',myfishtest[-1])
    

heart Y/Y   N/Y
      Y/N   N/N
        brain
c14.MEF2A c107-POU/2
     2   48
     113   2677
p = 1.0
c14.MEF2A c65-NEURO/OLIG
     3   47
     51   2739
p = 0.06804270132323588
c14.MEF2A c42.CUX1
     0   50
     18   2772
p = 1.0
c14.MEF2A c274-HNF1/POU
     8   42
     77   2713
p = 8.899930709973722e-05
c14.MEF2A c278-HD/5
     14   36
     561   2229
p = 0.211518219019046


In [32]:
reference_point = 'c70.MEF2A/2'
rowtalbe = []

testers = ['POU1F1_POU_1'.upper(), 'Atoh1.mouse_bHLH_1'.upper(), 'ONECUT3_CUT_1'.upper(), 'POU4F3_POU_1'.upper(), 'SRF_MADS_2'.upper(), 'Pou3f4_3773.1'.upper()]
print('heart Y/Y   N/Y')
print('      Y/N   N/N')
print('        brain')
for tester in testers:


    testcluster = clustDF_lookup.loc[tester].clust_name

    print(reference_point, testcluster)

    
    N_DHS_brain = mergedDF[mergedDF[testcluster] > 0].shape[0]
    N_DHS_total = mergedDF.shape[0]
    Nheart = mergedDF[mergedDF[reference_point] >0 ].shape[0]
        
    N_DHS_Brain_InHeart = mergedDF[(mergedDF[testcluster] > 0) & (mergedDF[reference_point] > 0)].shape[0]
    

    print('     '+str(N_DHS_Brain_InHeart)+'   '+str(Nheart-N_DHS_Brain_InHeart ))
    print('     '+str(N_DHS_brain- N_DHS_Brain_InHeart)+'   '+str(N_DHS_total - N_DHS_brain - Nheart + N_DHS_Brain_InHeart))
    myfishtest = fisher_exact([[N_DHS_Brain_InHeart,Nheart-N_DHS_Brain_InHeart  ], [N_DHS_brain- N_DHS_Brain_InHeart, N_DHS_total - N_DHS_brain - Nheart + N_DHS_Brain_InHeart]])
    print('p =',myfishtest[-1])
    

heart Y/Y   N/Y
      Y/N   N/N
        brain
c70.MEF2A/2 c107-POU/2
     9   164
     106   2561
p = 0.4229221086654831
c70.MEF2A/2 c65-NEURO/OLIG
     4   169
     50   2617
p = 0.5683772842006201
c70.MEF2A/2 c42.CUX1
     2   171
     16   2651
p = 0.3007409253700303
c70.MEF2A/2 c274-HNF1/POU
     11   162
     74   2593
p = 0.017168045276922686
c70.MEF2A/2 c55.SRF
     5   168
     65   2602
p = 0.6140298500091274
c70.MEF2A/2 c278-HD/5
     42   131
     533   2134
p = 0.17253987786626515
